In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Anxiety_disorder"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Anxiety_disorder/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Anxiety_disorder/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Anxiety_disorder/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Anxiety_disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os

# Step 1: Look for directories related to Anxiety disorder
tcga_subdirs = os.listdir(tcga_root_dir)
print(f"Available TCGA subdirectories: {tcga_subdirs}")

# Look for directory related to Anxiety disorder
# Anxiety disorder might be found as a comorbidity in neurological or brain-related cancers
# Examine if any directories might contain data relevant to anxiety disorders

# While anxiety is common in cancer patients, it's not a primary cancer type
# After reviewing all subdirectories, we need to determine if any datasets might contain 
# anxiety-related clinical information

# For this analysis, we'll use the brain cancer datasets as they may be more likely to 
# contain psychiatric comorbidity data
potential_matches = [
    'TCGA_Glioblastoma_(GBM)',
    'TCGA_Lower_Grade_Glioma_(LGG)',
    'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)'
]

# Select the most comprehensive dataset from potential matches
target_dir = 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)'
target_path = os.path.join(tcga_root_dir, target_dir)

print(f"Selected directory: {target_dir} - this dataset may contain clinical information about psychiatric conditions including anxiety")

# Step 2: Get the clinical and genetic data file paths
clinical_path, genetic_path = tcga_get_relevant_filepaths(target_path)

# Step 3: Load the datasets
clinical_df = pd.read_csv(clinical_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_path, sep='\t', index_col=0)

# Step 4: Print column names of clinical data
print("\nClinical data columns:")
print(clinical_df.columns.tolist())

# Check if we have both gene data and potential trait data
has_gene_data = not genetic_df.empty
has_potential_trait_data = not clinical_df.empty

# Record our initial assessment
validate_and_save_cohort_info(
    is_final=False, 
    cohort="TCGA", 
    info_path=json_path, 
    is_gene_available=has_gene_data, 
    is_trait_available=has_potential_trait_data
)


Available TCGA subdirectories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Canc


Clinical data columns:
['_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'animal_insect_allergy_history', 'animal_insect_allergy_types', 'asthma_history', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'days_to_additional_surgery_locoregional_procedure', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'days_to_performance_status_assessment', 'easte

False

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify candidate age and gender columns
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust', 
                     'first_diagnosis_age_of_animal_insect_allergy', 'first_diagnosis_age_of_food_allergy']
candidate_gender_cols = ['gender']

# Get the first TCGA dataset from the directory to examine the candidates
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)')
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the clinical data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract and preview age columns
age_preview = {}
if candidate_age_cols:
    for col in candidate_age_cols:
        if col in clinical_df.columns:
            age_preview[col] = clinical_df[col].head(5).tolist()

# Extract and preview gender columns
gender_preview = {}
if candidate_gender_cols:
    for col in candidate_gender_cols:
        if col in clinical_df.columns:
            gender_preview[col] = clinical_df[col].head(5).tolist()

print("Age columns preview:")
print(age_preview)
print("\nGender columns preview:")
print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [44.0, 50.0, 59.0, 56.0, 40.0], 'days_to_birth': [-16179.0, -18341.0, -21617.0, -20516.0, -14806.0], 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust': [nan, nan, nan, nan, nan], 'first_diagnosis_age_of_animal_insect_allergy': [nan, nan, nan, nan, nan], 'first_diagnosis_age_of_food_allergy': [nan, nan, nan, nan, nan]}

Gender columns preview:
{'gender': ['FEMALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Selecting the most appropriate age column
age_columns = {
    'age_at_initial_pathologic_diagnosis': [44.0, 50.0, 59.0, 56.0, 40.0], 
    'days_to_birth': [-16179.0, -18341.0, -21617.0, -20516.0, -14806.0], 
    'first_diagnosis_age_asth_ecz_hay_fev_mold_dust': [float('nan'), float('nan'), float('nan'), float('nan'), float('nan')], 
    'first_diagnosis_age_of_animal_insect_allergy': [float('nan'), float('nan'), float('nan'), float('nan'), float('nan')], 
    'first_diagnosis_age_of_food_allergy': [float('nan'), float('nan'), float('nan'), float('nan'), float('nan')]
}

# 'age_at_initial_pathologic_diagnosis' has meaningful values with no missing values
age_col = 'age_at_initial_pathologic_diagnosis'

# Selecting the most appropriate gender column
gender_columns = {
    'gender': ['FEMALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']
}

# 'gender' is the only column and has meaningful values
gender_col = 'gender'

# Print the chosen columns
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize the clinical features
# Get file paths - use the brain cancer dataset identified earlier
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)')
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Look for any anxiety-related columns in the clinical data
anxiety_related_cols = [col for col in clinical_df.columns if any(term in col.lower() for term in 
                       ['anxiety', 'mental', 'psychiatric', 'psychological', 'mood'])]
print(f"Potential anxiety-related columns: {anxiety_related_cols}")

# Check for other columns that might indirectly relate to anxiety
other_relevant_cols = ['mental_status_changes', 'seizure_history', 'headache_history']
existing_relevant_cols = [col for col in other_relevant_cols if col in clinical_df.columns]
print(f"Other potentially relevant columns: {existing_relevant_cols}")

# Examine these columns if they exist
for col in existing_relevant_cols:
    print(f"\nValues in {col}:")
    print(clinical_df[col].value_counts())

# While mental_status_changes exists, it's not a specific indicator of anxiety disorder
print("\nNo direct anxiety disorder indicator found in the TCGA dataset")

# Set flags to indicate that anxiety disorder trait is not available
is_gene_available = True  # We do have gene data
is_trait_available = False  # But we don't have anxiety disorder data

# Validate and save this information - use is_final=False since we're just recording unavailability
is_usable = validate_and_save_cohort_info(
    is_final=False,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

print(f"Dataset usability status: {is_usable}")
print("Processing completed. No data saved as anxiety disorder information is not available in TCGA datasets.")

Potential anxiety-related columns: ['mental_status_changes']
Other potentially relevant columns: ['mental_status_changes', 'seizure_history', 'headache_history']

Values in mental_status_changes:
mental_status_changes
NO     353
YES    120
Name: count, dtype: int64

Values in seizure_history:
seizure_history
YES    311
NO     183
Name: count, dtype: int64

Values in headache_history:
headache_history
NO     302
YES    177
Name: count, dtype: int64

No direct anxiety disorder indicator found in the TCGA dataset
Dataset usability status: False
Processing completed. No data saved as anxiety disorder information is not available in TCGA datasets.
